In [289]:
from TM1py.Services import TM1Service
from TM1py.Utils import Utils
import pandas as pd
from sklearn.linear_model import LinearRegression
import time
import matplotlib.pyplot as plt
import re
from typing import List

### 获取Training 实际数部分

In [2]:
zl_report_all=pd.read_csv(r'C:\TM1py\zl_amazon_payments_data_report.csv',
                   dtype={'SA Year':'str','Order Year':'str','COA Company':'str','Platform':'str'})

In [4]:
zl_report_all

,Scenario,SA Year,SA Month,Version,COA Company,Currency,Country,Product,Platform,ZL Amazon Management Report Item,Data Source Payments Data Report,M ZL Amazon Payments Data Report,Value
0,ACT,2018,M01,WIP,002,USD,US,LSKU1787,07,AMZ SP广告费分摊,Base,Amount,278.9424
1,ACT,2018,M01,WIP,002,USD,US,LSKU1787,07,AMZ SD广告费分摊,Base,Amount,43.1232
2,ACT,2018,M01,WIP,002,USD,US,LSKU1284,07,AMZ SP广告费分摊,Base,Amount,161.3496
3,ACT,2018,M01,WIP,002,USD,US,LSKU1284,07,AMZ SD广告费分摊,Base,Amount,0.2784
4,ACT,2018,M01,WIP,002,USD,US,LSKU1931,07,AMZ SP广告费分摊,Base,Amount,79.6800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6706997,Forecast,2021,M12,WIP,002,No Currency,SE,LSKU1179,05,销售数量(不含退货),Base,Amount,1.0000
6706998,Forecast,2021,M12,WIP,002,No Currency,SE,LSKU1904,05,退货数量,Base,Amount,-1.0000
6706999,Forecast,2021,M12,WIP,002,No Currency,SE,LSKU1303,05,销售数量(不含退货),Base,Amount,1.0000
6707000,Forecast,2021,M12,WIP,002,No Currency,TR,LSKU1929,06,销售数量(不含退货),Base,Amount,1.0000


### 特征列与预测销量的回归

#### 对一个单月在多个年度的数据进行了回归（回归了12个单个月的回归模型）

In [17]:
import pandas as pd
import asyncio
import nest_asyncio
nest_asyncio.apply()
from concurrent.futures import ThreadPoolExecutor
from typing import List
from TM1py.Services import TM1Service

In [49]:
dict_linearRegress=dict()

async def Linear_async(df:'pd.DataFrame',months:List):
    loop = asyncio.get_event_loop()
    
    with ThreadPoolExecutor(8) as executor:
        #futures = [loop.rudata:'pd.DataFrame',n_in_executor(executor, transfer, tm1_src, tm1_tgr,cube_src, cube_tgr,month_leaf) for month_leaf in leaves]
        futures=[]
        for month in months:
            
            f=loop.run_in_executor(executor, linear_model_ACT,df,month)
            futures.append(f)
        for future in futures:
            try:
                await future
            except:
                print(future)
                continue      

def linear_model_ACT(df:'pd.DataFrame',month:str):
    def transfer_c(df:'pd.DataFrame',month:str):

        df_filter=df[(df['Scenario']=='ACT')&(df['SA Month']==month)&(df['Currency']=='USD') 
                      &((df['ZL Amazon Management Report Item']=='销售数量(不含退货)')
                        |(df['ZL Amazon Management Report Item']=='AMZ SP广告费分摊')
                        |(df['ZL Amazon Management Report Item']=='AMZ SD广告费分摊')
                        |(df['ZL Amazon Management Report Item']=='AMZ SBV广告费分摊')
                        |(df['ZL Amazon Management Report Item']=='AMZ SB广告费分摊')
                        |(df['ZL Amazon Management Report Item']=='AMZ DSP广告费分摊'))]
        df_pivot=df_filter.pivot(index=['Scenario','SA Year','SA Month','Version','COA Company','Currency','Country','Product','Platform','Data Source Payments Data Report','M ZL Amazon Payments Data Report']
                            ,columns='ZL Amazon Management Report Item',values='Value').reset_index().reset_index(drop=True)
        return  df_pivot
    df=transfer_c(zl_report_all,month)
    #获取实际数数据df去除NA
    df.dropna(inplace=True)
    #日期处理
    df['yearmonth']=df['SA Year']+'-'+df['SA Month'].str.slice(1,3)
    df.loc[:,'yearmonth']=pd.to_datetime(df['yearmonth'])
    df.loc[:,'yearmonth']= df['yearmonth'].apply(lambda x:time.mktime(x.timetuple()))
    #自变量定义
    Z=df[['AMZ SP广告费分摊','AMZ SD广告费分摊','AMZ SBV广告费分摊','AMZ SB广告费分摊','AMZ DSP广告费分摊','yearmonth']]
    #回归
    lr=LinearRegression()
    lr.fit(Z,df['销售数量(不含退货)'])
    print('R Square:',lr.score(Z,df['销售数量(不含退货)']))
    print('intercept:',lr.intercept_)
    print('coef',lr.coef_)
    dict_linearRegress[month]=lr
    return lr 

In [50]:
months_list=['M01','M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12']
asyncio.run(Linear_async(zl_report_all,months_list))

R Square: R Square:R Square: 0.87487585631682530.6657360561404913
intercept:
intercept: 7915.0354246485385R Square: 0.8206118116178676
 
 1894.2313605071047
0.8388148106037041
intercept: 1865.924351998988intercept:coef [-8.29723964e-04 -7.93194499e-02 -4.45220629e-02  2.11855934e-02
  7.06926754e-01 -1.12455858e-06]

coef [ 8.58634305e-02 -3.21055807e-01 -1.20195073e-01  2.93934601e-01
  7.10904206e-01 -4.93021866e-06]
coef  [-1.82095717e-01 -3.48502906e-02 -1.29239595e-01  1.13307275e-01
  1.04961615e+00 -1.09861549e-06]
9211.201942204714
coef [ 1.57578132e-01 -3.83506367e-01 -2.70392407e-02  1.71107989e-01
  4.68717372e-01 -5.74490069e-06]
R Square: 0.7653106316922222
intercept: 4442.407755315352
coefR Square: 0.8054712032014497
intercept: 2775.622266638648
coef [-9.15833671e-02 -1.54187730e-01 -1.42575641e-01  6.33199395e-02
  8.27464683e-01 -1.61584790e-06]
 [-5.55944968e-02  3.04245800e-02 -1.10041047e-01  5.01738271e-02
  7.92530791e-01 -2.68176307e-06]
R Square: 0.70807173864561

#### 连接Tm1

In [1]:
tm1 = TM1Service(address="192.168.0.176", port=30015, ssl=False, user="neil", password="123")

NameError: name 'TM1Service' is not defined

#### 处理预测填报的数据

In [153]:
#处理目标cube的数据
def transfer_fcst(tm1: TM1Service,month:str):
        mdx='''
        SELECT 
          NON EMPTY 
           {Tm1filterbylevel({[ZL AMZ Fee Item].[ZL AMZ Fee Item].Members} ,0)}
            ON COLUMNS , 
          NON EMPTY 
           {Tm1filterbylevel({[Product].[Product].Members},0)}
           *{[SA Month].[SA Month].['''+month+''']}
           *{Tm1filterbylevel( {[Platform].[Platform].Members},0)}
           *{Tm1filterbylevel( {[Fee Allocation Method].[Fee Allocation Method].Members} ,0)}  
            ON ROWS 
        FROM [Sys Definition ZL AMZ Fee STG] 
        WHERE 
          (
           [Scenario].[Scenario].[FCST M01 CY2022],
           [SA Year].[SA Year].[2022],
           [COA Company].[COA Company].[002],
           [Version].[Version].[WIP],
           [Data Source ZL AMZ Fee STG].[Data Source ZL AMZ Fee STG].[Base],
           [M Sys Definition ZL AMZ Fee STG].[M Sys Definition ZL AMZ Fee STG].[Amount],
           [Currency].[Currency].[USD],
           [Country].[Country].[US]
          )
             '''
        #元组字典
        cells=tm1.cubes.cells.execute_mdx(mdx,skip_rule_derived_cells=True,skip_zeros=True,skip_consolidated_cells=True)
        df_nlevel = Utils.build_pandas_dataframe_from_cellset(cells)
        df_nlevel=df_nlevel.reset_index()
        df_nlevel.loc[:,'ZL AMZ Fee Item']=df_nlevel['ZL AMZ Fee Item'].str.replace('Sponsored Products','AMZ SP广告费分摊')
        df_nlevel.loc[:,'ZL AMZ Fee Item']=df_nlevel['ZL AMZ Fee Item'].str.replace('Sponsored Display','AMZ SD广告费分摊')
        df_nlevel.loc[:,'ZL AMZ Fee Item']=df_nlevel['ZL AMZ Fee Item'].str.replace('Sponsored Brands videos','AMZ SBV广告费分摊')
        df_nlevel.loc[:,'ZL AMZ Fee Item']=df_nlevel['ZL AMZ Fee Item'].str.replace('Sponsored Brands','AMZ SB广告费分摊')
        df_nlevel.loc[:,'ZL AMZ Fee Item']=df_nlevel['ZL AMZ Fee Item'].str.replace('Demand-Side Platform','AMZ DSP广告费分摊')
        df_pivot_n=df_nlevel.pivot(index=['Scenario','SA Year','SA Month','Version','COA Company','Currency','Country','Platform','Product','Fee Allocation Method'
                              ,'Data Source ZL AMZ Fee STG','M Sys Definition ZL AMZ Fee STG'],columns='ZL AMZ Fee Item'
                      ,values='Values').reset_index().reset_index(drop=True)
        #插值法
        from scipy.interpolate import lagrange
        def lag_inter(s):
            for n in range(0,len(s)):
                if s.isnull()[n]:
                    #y=s.iloc[(n-150):(n+150+150),:]
                    #y.dropna(inplace=True)
                    #v=lagrange(y.index,list(y))(n)
                    s.iloc[n,:]=0
        lag_inter(df_pivot_n['AMZ DSP广告费分摊'])
        lag_inter(df_pivot_n['AMZ SB广告费分摊'])
        lag_inter(df_pivot_n['AMZ SBV广告费分摊'])
        lag_inter(df_pivot_n['AMZ SD广告费分摊'])
        lag_inter(df_pivot_n['AMZ SP广告费分摊'])
        df_pivot_n.dropna(inplace=True)
        return df_pivot_n

##### 处理某维度中包含的特征列，将之透视变成真正的平铺的列的样式，方便回归

In [226]:
df_fcst=transfer_fcst(tm1,'M01')

C:\Users\User\AppData\Roaming\Python\Python36\site-packages\pandas\core\indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [227]:
df_fcst

ZL AMZ Fee Item,Scenario,SA Year,SA Month,Version,COA Company,Currency,Country,Platform,Product,Fee Allocation Method,Data Source ZL AMZ Fee STG,M Sys Definition ZL AMZ Fee STG,AMZ DSP广告费分摊,AMZ SBV广告费分摊,AMZ SB广告费分摊,AMZ SD广告费分摊,AMZ SP广告费分摊
0,FCST M01 CY2022,2022,M01,WIP,002,USD,US,01,LSKU1404,Allocation by Revenue,Base,Amount,33.522205,0.0,0.0,0.0,0.00000
1,FCST M01 CY2022,2022,M01,WIP,002,USD,US,01,LSKU1410,Allocation by Revenue,Base,Amount,36.383722,0.0,0.0,0.0,0.00000
2,FCST M01 CY2022,2022,M01,WIP,002,USD,US,01,LSKU1411,Allocation by MSKU,Base,Amount,0.000000,0.0,0.0,0.0,120.27004
3,FCST M01 CY2022,2022,M01,WIP,002,USD,US,01,LSKU1411,Allocation by Revenue,Base,Amount,23.005709,0.0,0.0,0.0,0.00000
4,FCST M01 CY2022,2022,M01,WIP,002,USD,US,01,LSKU1414,Allocation by Revenue,Base,Amount,0.300306,0.0,0.0,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1288,FCST M01 CY2022,2022,M01,WIP,002,USD,US,15,LSKU1202,Allocation by MSKU,Base,Amount,0.000000,0.0,0.0,0.0,566.49846
1289,FCST M01 CY2022,2022,M01,WIP,002,USD,US,15,LSKU1332,Allocation by MSKU,Base,Amount,0.000000,0.0,0.0,0.0,109.76218
1290,FCST M01 CY2022,2022,M01,WIP,002,USD,US,15,LSKU1333,Allocation by MSKU,Base,Amount,0.000000,0.0,0.0,0.0,125.24886
1291,FCST M01 CY2022,2022,M01,WIP,002,USD,US,15,LSKU1334,Allocation by MSKU,Base,Amount,0.000000,0.0,0.0,0.0,85.13648


### 预测

#### 将数据处理成Tm1可识别的逆透视样式

In [285]:
def linear_model_FCST(df:'pd.DataFrame',lr:'LinearRegress list',month:str):
    #处理目标cube的日期
    df['yearmonth']=df['SA Year']+'-'+df['SA Month'].str.slice(1,3)
    df.loc[:,'yearmonth']=pd.to_datetime(df['yearmonth'])
    df.loc[:,'yearmonth']= df['yearmonth'].apply(lambda x:time.mktime(x.timetuple()))
    #回归预测
    Z_tgr=df[['AMZ SP广告费分摊','AMZ SD广告费分摊','AMZ SBV广告费分摊','AMZ SB广告费分摊','AMZ DSP广告费分摊','yearmonth']]
    y_predict=lr[month].predict(Z_tgr)
    df['Sales Units']=y_predict
    df.drop(['yearmonth'],axis=1,inplace=True)
    df_ready=df.melt(id_vars=['Scenario','SA Year','SA Month','Version','COA Company','Currency','Country','Platform','Product','Fee Allocation Method'
                        ,'Data Source ZL AMZ Fee STG','M Sys Definition ZL AMZ Fee STG'],var_name=['ZL AMZ Fee Item'],value_name='Values')
    #获取待回写的Dataframe   d
    return df_ready   

### 预测并在Tm1中写入数据

In [286]:
async def fcst_async(tm1:TM1Service,lr:'LinearRegress list',months:List,cube_tgr:str):
    loop = asyncio.get_event_loop()
    
    with ThreadPoolExecutor(8) as executor:
        #futures = [loop.rudata:'pd.DataFrame',n_in_executor(executor, transfer, tm1_src, tm1_tgr,cube_src, cube_tgr,month_leaf) for month_leaf in leaves]
        futures=[]
        for month in months:
            
            f=loop.run_in_executor(executor, linear_transfer,tm1,lr,month,cube_tgr)
            futures.append(f)
        for future in futures:
            try:
                await future
            except:
                print(future)
                continue  



def linear_transfer(tm1:TM1Service,lr:'LinearRegress list',month:str,cube_tgr:str):
    
    df=transfer_fcst(tm1,month)
    
    df=linear_model_FCST(df,lr,month)
    df['Business Unit']='00501'
    df['Customer']='No Amazon Customer'
    df['M ZL Product Sales Plan']='Sales Units'
    df['Data Source ZL Product Sales Plan']='Linear FCST'
    df['Channel']='Amazon'
    df=df[df['ZL AMZ Fee Item']=='Sales Units']
    df.drop(['Platform','Fee Allocation Method','Data Source ZL AMZ Fee STG','M Sys Definition ZL AMZ Fee STG','ZL AMZ Fee Item'],axis=1,inplace=True)
    df.rename(columns={'Country':'Region','ZL AMZ Fee Item':'M ZL Product Sales Plan'},inplace=True)
    col_name=['Scenario','SA Year','SA Month','Version','COA Company','Business Unit','Product','Currency','Region','Channel','Customer','Data Source ZL Product Sales Plan','M ZL Product Sales Plan','Values']
    df_final=df.reindex(columns=col_name)
    #clear
    mdx_tgr='''
        SELECT 
           {Tm1filterbylevel({[Region].[Region].Members},0)}
           *{[SA Month].[SA Month].['''+month+''']}
          ON COLUMNS , 
           {Tm1filterbylevel( {[Product].[Product].Members},0)}
           *{Tm1filterbylevel( {[Customer].[Customer].Members},0)}
           ON ROWS 
        FROM [ZL Product Sales Plan] 
        WHERE 
          (
           [Scenario].[Scenario].[FCST M01 CY2022],
           [SA Year].[SA Year].[2022],
           [Version].[Version].[WIP],
           [COA Company].[COA Company].[002],
           [Data Source ZL Product Sales Plan].[Data Source ZL Product Sales Plan].[Linear FCST],
           [Currency].[Currency].[USD],
           [M ZL Product Sales Plan].[M ZL Product Sales Plan].[Sales Units],
           [Business Unit].[Business Unit].[00501],
           [Channel].[Channel].[Amazon]
  )
             '''
    tm1.cells.clear_with_mdx(cube_tgr,mdx_tgr)
    #Writing to tgr Cube
    tm1.cells.write_dataframe(
        cube_name=cube_tgr,
        data=df_final,
        dimensions=['Scenario','SA Year','SA Month','Version','COA Company','Business Unit','Product','Currency'
                    ,'Region','Channel','Customer','Data Source ZL Product Sales Plan','M ZL Product Sales Plan'],
        increment=True,
        deactivate_transaction_log=True,
        reactivate_transaction_log=True,
        sandbox_name=None,
        use_ti=True,
        skip_non_updateable=False,
        use_changeset=False)

In [288]:
months_list=['M01','M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12']
asyncio.run(fcst_async(tm1,dict_linearRegress,months_list,'ZL Product Sales Plan'))

C:\Users\User\AppData\Roaming\Python\Python36\site-packages\pandas\core\indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\Users\User\AppData\Roaming\Python\Python36\site-packages\pandas\core\indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\Users\User\AppData\Roaming\Python\Python36\site-packages\pandas\core\indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d